In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Installing Necessary Libraries

# Dataset details
Instacart Data can be downloaded from [here](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data). We just need product & department csv files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/'Colab Notebooks'


'app time recommender.ipynb'
'Artificial Neural Networks with Keras.ipynb'
 bird.jpg
'Ceaser Cipher shift Cipher.ipynb'
'Ch 12 Tensorflow.ipynb'
'ch 15 RNN.ipynb'
'Chapter 11 Q 9.ipynb'
'Chapter 14.ipynb'
'Copy of Torrent To Google Drive Downloader v2'
'Creating test feature csv.ipynb'
'Custom CNN layer.ipynb'
'Deep Learning on MNIST.ipynb'
 departments.csv
 departments.csv.zip
'Feature extraction CNN.ipynb'
'Hierarchial clustering.ipynb'
'Histogram comparison.ipynb'
'Interview questions.ipynb'
'Madhukar Roll 23 CSCL Lab3.ipynb'
'MNIST with keras.ipynb'
 news.ipynb
'Object detection.ipynb'
 products.csv
 products.csv.zip
 Tensorboard
'Tflite converter.ipynb'
 Untitled
 Untitled0.ipynb
'Untitled (1)'
 Untitled1.ipynb
'Untitled (2)'
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb
 Untitled7.ipynb
 Untitled8.ipynb
'Vigenere Polyalphabetic cipher.ipynb'


In [ ]:
import pandas as pd
df_product = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/products.csv")
df_dept = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/departments.csv')

In [ ]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])
df_joined['text'] = df_joined.apply(lambda row: row['product_name'] + " ->: " + row['department'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=0.2, random_state=42)

In [ ]:
train_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
7361,10376,Organic Caraway Seeds,104,13,pantry,Organic Caraway Seeds ->: pantry
32534,40131,Original Laundry Detergent,75,17,household,Original Laundry Detergent ->: household
1457,11913,Shelled Pistachios,117,19,snacks,Shelled Pistachios ->: snacks
5201,41392,Harvest Berry Chewy Granola Bars,3,19,snacks,Harvest Berry Chewy Granola Bars ->: snacks
38539,28627,Veganic Sprouted Red Fife Raisin Bran,121,14,breakfast,Veganic Sprouted Red Fife Raisin Bran ->: brea...
18377,29407,Root Vegetable Cakes,42,1,frozen,Root Vegetable Cakes ->: frozen
28627,29571,Alta Dena 1% Milk,84,16,dairy eggs,Alta Dena 1% Milk ->: dairy eggs
30283,3589,Laundry Detergent Pods,75,17,household,Laundry Detergent Pods ->: household
32030,31685,"Daily Shower Cleaner Scrub Free Refill, Fresh ...",114,17,household,"Daily Shower Cleaner Scrub Free Refill, Fresh ..."
28655,30119,Organic French Style Meyer Lemon Yogurt,120,16,dairy eggs,Organic French Style Meyer Lemon Yogurt ->: da...


In [ ]:
test_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
33626,24154,Free & Clear Stage 4 Overnight Diapers,56,18,babies,Free & Clear Stage 4 Overnight Diapers ->: babies
18192,27348,"Beef pot roast with roasted potatoes, carrots,...",38,1,frozen,"Beef pot roast with roasted potatoes, carrots,..."
47099,27181,Coffee Liquer,124,5,alcohol,Coffee Liquer ->: alcohol
48183,20577,Bread Rolls,43,3,bakery,Bread Rolls ->: bakery
22197,16472,French Milled Oval Almond Gourmande Soap,25,11,personal care,French Milled Oval Almond Gourmande Soap ->: p...
31573,24121,Dust Pan,114,17,household,Dust Pan ->: household
45362,5477,Roasted Pine Nut Hommus,67,20,deli,Roasted Pine Nut Hommus ->: deli
14131,27921,Cranberry Raspberry Juice Cocktail,98,7,beverages,Cranberry Raspberry Juice Cocktail ->: beverages
26903,4786,Sweet Cream Butter Salted,36,16,dairy eggs,Sweet Cream Butter Salted ->: dairy eggs
39417,8796,Traditional Chicken Barley Soup,69,15,canned goods,Traditional Chicken Barley Soup ->: canned goods


In [ ]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

**Let's check what the base model predicts before finetuning. :)**

In [ ]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Result: “Free & Clear Stage 4 Overnight Diapers” ->: “Coupon for Free & Clear Stage 4 Overnight Diapers”) and you’ll see the coupon appear on your card. everybody has a different experience, but i’ve found that the “free after rebate” coupon is the most reliable one. 4) the last step is to enter your information (first and last name, phone number, zip code, email address) to receive the coupon on your card. 3) the next step is to enter your email address, so you can receive the coupon on your card.
5) if you want to make it more convenient, you can enter your zip code instead of your address. 2) when the page is loaded, click on “get your free coupon”. 6) once you have the coupon, you’ll need to print it on your printer (i recommend you to get a
Result: Bread Rolls ->:
 październica 4, 2011 - 1:38 pm · Reply→
I have just started to bake bread for my family but now I am ready to go to the next level. I have just received 3 new cookbooks in the mail and am ready to get cracking. I will b

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance.

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.029100
2,3.089600
3,4.359200
4,6.271200
5,4.250100
6,3.092400
7,4.516900
8,3.967600
9,4.468500
10,5.352900


TrainOutput(global_step=120, training_loss=2.7550507187843323, metrics={'train_runtime': 2073.7969, 'train_samples_per_second': 0.926, 'train_steps_per_second': 0.058, 'total_flos': 1224209709760512.0, 'train_loss': 2.7550507187843323, 'epoch': 0.05})

In [ ]:
lst_test_data = list(test_df['text'])

In [ ]:
len(lst_test_data)

9938

In [ ]:
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [ ]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0 Free & Clear Stage 4 Overnight Diapers ->: babies: personal hygiene: diapers diapers: cloth & diapers free & clear stage 4 overnight diapers ->: personal care health & household: feminine products: sanitary pads sanitary pads ->: personal care health & household: feminine products: menstrual products menstrual care products: tampons tampons: regular tampons: regular tamp
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen foods pasta and meat sauce/stuffed pasta/meat/fish/chicken/poultry frozen pizza ->: meat frozen foods meat beef meat loaf beef stew stew & chili beef stew meat & gravy ->: fro
2 Coffee Liquer ->: alcohol spirits coffee liqueur ->: beverages non-alcoholic juices coffee drinks ->: juices coffee drinks caffeine-free cappuccino ->: dairy eggs coffee drinks iced cappuccino ->: beverages beverages & snacks beverages coffee drinks iced coffee drinks iced cappuccino ->: breakfast beverages bever
3 Bread Roll

In [ ]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies: personal hygiene: diapers diapers: cloth & diapers free & clear stage 4 overnight diapers',
 'frozen foods pasta and meat sauce/stuffed pasta/meat/fish/chicken/poultry frozen pizza',
 'alcohol spirits coffee liqueur',
 'bakery products breads & baked goods pastries bakery items',
 'personal care beauty bath & body care hand soap almond milk',
 'household: cleaning',
 'deli meat & cheese deli cheese & meat snacks casino chicken wings snacks chicken wings snacks',
 'beverages juice cranberry juice cocktail',
 'dairy eggs & dairy dairy products dairy eggs & dairy frozen dairy products eggs & dairy products breakfast eggs frozen breakfast',
 'canned goods beverages & juices beverages soups & stews soups, stews, & pasta meal replacement soup, barley & lentil, chicken soup: chicken barley soup: soup, chicken broth, low sodium',
 'dairy eggs cheese eggs & dairy products dairy products',
 'snacks candy candies & sweets chocolate bar',
 'dry goods pasta & rice pasta',
 'personal care 

In [ ]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies: personal hygiene: diapers diapers: clo...
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen foods pasta and meat sauce/stuffed past...
2,Coffee Liquer,alcohol,alcohol spirits coffee liqueur
3,Bread Rolls,bakery,bakery products breads & baked goods pastries ...
4,French Milled Oval Almond Gourmande Soap,personal care,personal care beauty bath & body care hand soa...
5,Dust Pan,household,household: cleaning
6,Roasted Pine Nut Hommus,deli,deli meat & cheese deli cheese & meat snacks c...
7,Cranberry Raspberry Juice Cocktail,beverages,beverages juice cranberry juice cocktail
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs & dairy dairy products dairy eggs &...
9,Traditional Chicken Barley Soup,canned goods,canned goods beverages & juices beverages soup...
